In [1]:
!pip install optuna

  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11107 sha256=e4e3c2ee8e1e2ab8a579ed074434ab1740718d5634be13eceb82e224cac2b5c6
  Stored in directory: c:\users\pfsl\appdata\local\pip\cache\wheels\7f\1a\65\84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [2]:
import optuna

In [3]:
!pip install xgboost

In [4]:
import xgboost as xgb

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv("Admission_Predict.csv", sep=",")

In [8]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


In [20]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mean())

In [21]:
x = df.drop(columns=['Serial No.', 'Chance of Admit '], axis=1)
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
395,324,110,3,3.5,3.5,9.04,1
396,325,107,3,3.0,3.5,9.11,1
397,330,116,4,5.0,4.5,9.45,1
398,312,103,3,3.5,4.0,8.78,0


In [25]:
x.describe()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500
std,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362
min,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000
25%,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000
50%,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000
75%,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000
max,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000


In [22]:
y = df[['Chance of Admit ']]
y

,Chance of Admit
0,0.92
1,0.76
2,0.72
3,0.80
4,0.65
...,...
395,0.82
396,0.84
397,0.91
398,0.67


In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled_arr = scaler.fit_transform(x)

In [35]:
# tree_method is a parameter in XGBoost library which has values as gpu_hit and it will increase the computation

from sklearn.metrics import mean_squared_error

def objective(trials, data = x, target = y):
    x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.10, random_state=30)
    
    param = {
        #'tree_method' : 'gpu_hist', # commented because there is no GPU
        'lambda' : trials.suggest_loguniform('lambda', 1e-4, 10.0),
        'alpha' : trials.suggest_loguniform('alpha', 1e-4, 10.0),
        'colsample_bytree' : trials.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
        'subsample' : trials.suggest_categorical('subsample', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
        'learning_rate' : trials.suggest_categorical('learning_rate', [0.001,0.003,0.005,0.008,0.02,0.01,1,10,20]),
        'n_estimator' : trials.suggest_categorical('n_estimator', [1,5,10,20,20,50,100]),
        'max_depth' : trials.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10]),
        'random_state' : trials.suggest_categorical('random_state', [10,20,30,50,100]),
        'min_child_weight' : trials.suggest_int('min_child_weight', 1, 200)
    }
    
    xgb_reg_model = xgb.XGBRFRegressor(**param)
    xgb_reg_model.fit(x_train, y_train, eval_set=[(x_test, y_test)])
    pred_xgb = xgb_reg_model.predict(x_test)
    rmse = mean_squared_error(y_test, pred_xgb)
    
    return rmse

In [39]:
find_optuna_params = optuna.create_study(direction='minimize')
find_optuna_params.optimize(objective, n_trials = 100)
find_optuna_params.best_trial.params

[I 2021-10-31 16:23:26,803] A new study created in memory with name: no-name-36d8a664-ba81-42a6-9a15-5cdecef760d7


[16:23:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:0.05670


[I 2021-10-31 16:23:26,937] Trial 0 finished with value: 0.003214867990237965 and parameters: {'lambda': 5.384831818412256, 'alpha': 0.22233643404876388, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 1, 'n_estimator': 50, 'max_depth': 7, 'random_state': 50, 'min_child_weight': 32}. Best is trial 0 with value: 0.003214867990237965.


[16:23:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:0.14963


[I 2021-10-31 16:23:26,996] Trial 1 finished with value: 0.022389471214641996 and parameters: {'lambda': 0.0009513827518950735, 'alpha': 0.005259016262925875, 'colsample_bytree': 0.4, 'subsample': 0.6, 'learning_rate': 1, 'n_estimator': 100, 'max_depth': 7, 'random_state': 20, 'min_child_weight': 168}. Best is trial 0 with value: 0.003214867990237965.


...

[16:23:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:0.05112


[I 2021-10-31 16:23:37,110] Trial 98 finished with value: 0.00261314036243274 and parameters: {'lambda': 0.00011102470375483974, 'alpha': 0.0006828287757915234, 'colsample_bytree': 0.8, 'subsample': 0.8, 'learning_rate': 1, 'n_estimator': 20, 'max_depth': 4, 'random_state': 20, 'min_child_weight': 5}. Best is trial 97 with value: 0.0025581892993408522.


[16:23:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:0.27540


[I 2021-10-31 16:23:37,218] Trial 99 finished with value: 0.07584623000174724 and parameters: {'lambda': 0.00012110821349295412, 'alpha': 0.0005472161227504711, 'colsample_bytree': 0.8, 'subsample': 0.4, 'learning_rate': 0.008, 'n_estimator': 20, 'max_depth': 4, 'random_state': 20, 'min_child_weight': 14}. Best is trial 97 with value: 0.0025581892993408522.


{'lambda': 0.00012552746220058055,
 'alpha': 0.06992223607515607,
 'colsample_bytree': 0.7,
 'subsample': 0.8,
 'learning_rate': 1,
 'n_estimator': 20,
 'max_depth': 4,
 'random_state': 20,
 'min_child_weight': 6}

In [41]:
find_optuna_params.best_params

{'lambda': 0.00012552746220058055,
 'alpha': 0.06992223607515607,
 'colsample_bytree': 0.7,
 'subsample': 0.8,
 'learning_rate': 1,
 'n_estimator': 20,
 'max_depth': 4,
 'random_state': 20,
 'min_child_weight': 6}

In [45]:
find_optuna_params.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_n_estimator,params_random_state,params_subsample,state
0,0,0.003215,2021-10-31 16:23:26.809049,2021-10-31 16:23:26.936609,0 days 00:00:00.127560,0.222336,0.7,5.384832,1.000,7,32,50,50,0.6,COMPLETE
1,1,0.022389,2021-10-31 16:23:26.939625,2021-10-31 16:23:26.996103,0 days 00:00:00.056478,0.005259,0.4,0.000951,1.000,7,168,100,20,0.6,COMPLETE
2,2,0.076178,2021-10-31 16:23:26.997967,2021-10-31 16:23:27.066770,0 days 00:00:00.068803,0.040703,0.9,0.001453,0.008,3,148,5,50,0.6,COMPLETE
3,3,0.075929,2021-10-31 16:23:27.070746,2021-10-31 16:23:27.150477,0 days 00:00:00.079731,5.794721,0.4,0.416551,0.010,4,118,20,50,1.0,COMPLETE
4,4,8.847875,2021-10-31 16:23:27.153472,2021-10-31 16:23:27.249206,0 days 00:00:00.095734,0.000357,0.4,0.235921,20.000,10,103,20,30,0.3,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,24.123465,2021-10-31 16:23:36.659802,2021-10-31 16:23:36.761497,0 days 00:00:00.101695,0.001252,0.3,0.000177,20.000,4,6,20,100,0.8,COMPLETE
96,96,0.002930,2021-10-31 16:23:36.763496,2021-10-31 16:23:36.875768,0 days 00:00:00.112272,0.039259,0.5,0.000256,1.000,4,14,20,100,0.8,COMPLETE
97,97,0.002558,2021-10-31 16:23:36.877793,2021-10-31 16:23:36.986568,0 days 00:00:00.108775,0.069922,0.7,0.000126,1.000,4,6,20,20,0.8,COMPLETE
98,98,0.002613,2021-10-31 16:23:36.988563,2021-10-31 16:23:37.109035,0 days 00:00:00.120472,0.000683,0.8,0.000111,1.000,4,5,20,20,0.8,COMPLETE


In [57]:
!pip install plotly

In [60]:
# Optimization achieved

import plotly

optuna.visualization.plot_optimization_history(find_optuna_params.best_params)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

# Model Training

In [49]:
find_optuna_params.best_params

{'lambda': 0.00012552746220058055,
 'alpha': 0.06992223607515607,
 'colsample_bytree': 0.7,
 'subsample': 0.8,
 'learning_rate': 1,
 'n_estimator': 20,
 'max_depth': 4,
 'random_state': 20,
 'min_child_weight': 6}

In [56]:
xgb_new  = xgb.XGBRFRegressor(lambda = 0.00012552746220058055,
 alpha=0.06992223607515607,
 colsample_bytree= 0.7,
 subsample= 0.8,
 learning_rate= 1,
 n_estimator= 20,
 max_depth= 4,
 random_state= 20,
 min_child_weight= 6)

SyntaxError: invalid syntax (<ipython-input-56-cc995b1ca607>, line 1)